In [1]:
import fitz
def pdf_extract(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text
ger = pdf_extract('germany/germany_7.pdf') + " " + pdf_extract('germany/germany_81.pdf') + " " + pdf_extract('germany/germany_82.pdf')+ " "+ pdf_extract('germany/germany_83.pdf')
spain = pdf_extract('spain/spain_7.pdf')+" "+pdf_extract('spain/spain_71.pdf')+" "+pdf_extract('spain/spain_72.pdf')+" "+pdf_extract('spain/spain_8.pdf')+" "+pdf_extract('spain/spain_9.pdf')
aus = pdf_extract('australia/australia.pdf')
phili = pdf_extract('phillipeans/Phili_7.pdf') +" "+pdf_extract('phillipeans/phili_8.pdf')+" "+pdf_extract('phillipeans/Phili_9.pdf')
uk = pdf_extract('UK/UK_7.pdf')+" "+pdf_extract('UK/UK_8.pdf')+" "+pdf_extract('UK/UK_9.pdf')

In [2]:
textbooks = [ger, spain, phili,uk, aus]

# Data Cleaning

In [3]:
import re
from langdetect import detect
from langdetect import LangDetectException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
def clean_text(post):
    if detect(post) != 'en':
        return ''
    # Make posts lowercase
    post = post.lower()

    # Remove punctuation
    post = re.sub(r'[^a-zA-Z\s]', ' ', post)
    
    # Remove words with repeated letters
    #post = re.sub(r'([a-zA-Z])\1+', r'\1', post)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(post)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    return ' '.join(filtered_text)

In [5]:
cleanTextbooks = []
for i in textbooks:
    cleanTextbooks.append(clean_text(i))

In [6]:
#These have spaces, we use these to select the male and female words
menWords = [" man ", " boy ", " male ", " brother ", " father ", " son ", " husband ", " king ", " prince ", " uncle ", " nephew ", " he ", " him ", " his ", " gentleman ", " sir ", " mr. ", " hero ", " lord ", " patriarch ", " men "]
womenWords = [" woman ", " girl ", " female ", " sister ", " mother ", " daughter ", " wife ", " queen ", " princess ", " aunt ", " niece ", " she ", " her ", " hers ", " lady ", " ma'am "," madam ", " mrs. ", " ms. ", " miss ", " heroine ", " dame ", " matriarch ", " women "]
#These have no spaces, so these can be used to throw away words during TFIDF
menWordsNoSpace = ["man", "boy", "male", "brother", "father", "son", "husband", "king", "prince", "uncle", "nephew", "he", "him", "his", "gentleman", "sir", "mr", "hero", "lord", "patriarch", "men"]
womenWordsNoSpace = ["woman", "girl", "female", "sister", "mother", "daughter", "wife", "queen", "princess", "aunt", "niece", "she", "her", "hers", "lady", "ma'am","madam", "mrs", "heroine", "dame", "matriarch", "women"]

In [26]:
menPara = [[] for _ in range(5)]
womenPara = [[] for _ in range(5)]
for i in range(0, 5):
    text = cleanTextbooks[i]
    paragraphs = [text[j:j+100] for j in range(0, len(text), 100)]
    for paragraph in paragraphs:
        for j in menWords:
            if j in paragraph:
                menPara[i].append(paragraph)
        for j in womenWords:
            if j in paragraph:
                womenPara[i].append(paragraph)

# Occurence

In [27]:
men_count = sum(len(sublist) for sublist in menPara)
print(men_count)
women_count = sum(len(sublist) for sublist in womenPara)
print(women_count)

1567
1078


# Firstness

In [28]:
femaleFirst = []
for sublist in womenPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1): 
                if words[i].lower() in womenWordsNoSpace and words[i + 1].lower() in menWordsNoSpace:
                    femaleFirst.append((words[i], words[i + 1]))
print(len(femaleFirst),femaleFirst)

45 [('aunt', 'uncle'), ('mother', 'father'), ('mother', 'father'), ('woman', 'hero'), ('girl', 'boy'), ('girl', 'boy'), ('aunt', 'uncle'), ('mother', 'father'), ('mother', 'father'), ('mother', 'father'), ('girl', 'brother'), ('lady', 'brother'), ('women', 'men'), ('mother', 'father'), ('mother', 'brother'), ('mother', 'mr'), ('wife', 'brother'), ('sister', 'son'), ('mother', 'son'), ('mother', 'son'), ('mother', 'son'), ('mother', 'son'), ('mother', 'son'), ('mother', 'son'), ('sister', 'brother'), ('daughter', 'father'), ('daughter', 'father'), ('princess', 'king'), ('princess', 'king'), ('mother', 'father'), ('girl', 'mr'), ('wife', 'boy'), ('mother', 'father'), ('sister', 'brother'), ('mother', 'father'), ('sister', 'husband'), ('sister', 'brother'), ('niece', 'nephew'), ('niece', 'nephew'), ('niece', 'brother'), ('niece', 'nephew'), ('wife', 'mr'), ('wife', 'mr'), ('wife', 'mr'), ('female', 'male')]


In [29]:
maleFirst = []
for sublist in menPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1):  
                if words[i].lower() in menWordsNoSpace and words[i + 1].lower() in womenWordsNoSpace:
                    maleFirst.append((words[i], words[i + 1]))
print(len(maleFirst),maleFirst)

71 [('men', 'women'), ('men', 'women'), ('brother', 'sister'), ('brother', 'sister'), ('brother', 'sister'), ('uncle', 'aunt'), ('uncle', 'aunt'), ('father', 'daughter'), ('father', 'daughter'), ('male', 'female'), ('man', 'woman'), ('man', 'woman'), ('nephew', 'niece'), ('brother', 'sister'), ('men', 'women'), ('brother', 'sister'), ('men', 'women'), ('uncle', 'aunt'), ('brother', 'sister'), ('man', 'woman'), ('man', 'woman'), ('men', 'women'), ('husband', 'wife'), ('husband', 'wife'), ('men', 'women'), ('men', 'women'), ('men', 'women'), ('son', 'daughter'), ('men', 'women'), ('men', 'women'), ('king', 'queen'), ('nephew', 'sister'), ('nephew', 'sister'), ('son', 'daughter'), ('father', 'daughter'), ('man', 'mrs'), ('father', 'mrs'), ('father', 'mrs'), ('man', 'woman'), ('sir', 'mrs'), ('sir', 'mrs'), ('male', 'female'), ('lord', 'lady'), ('lord', 'lady'), ('lord', 'lady'), ('king', 'queen'), ('king', 'queen'), ('king', 'queen'), ('lord', 'lady'), ('lord', 'lady'), ('lord', 'lady'), 

In [30]:
sameLevel = {
    "siblings": {"brother", "sister"},
    "parent_child": {"father", "mother", "son", "daughter"},
    "spouses": {"husband", "wife"},
    "royalty": {"king", "queen", "prince", "princess"},
    "extended_family": {"uncle", "aunt", "nephew", "niece"},
    "personal_pronouns": {"he", "she", "him", "her", "his", "hers"},
    "titles": {"gentleman", "lady", "sir", "ma'am", "mr", "mrs", "madam"},
    "heroic_roles": {"hero", "heroine"},
    "social_roles": {"patriarch", "matriarch"},
    "general_plural": {"men", "women"},
}
flat = {word for group in sameLevel.values() for word in group}
def isSame(pair):
    return pair[0] in flat and pair[1] in flat
femaleFirst = [pair for pair in femaleFirst if isSame(pair)]

In [31]:
maleFirst = [pair for pair in maleFirst if isSame(pair)]
print(len(maleFirst))
print(len(femaleFirst))

46
38


# Sentiment analysis

In [32]:
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
fscores = analyzer.polarity_scores(fw)
mscores = analyzer.polarity_scores(fm)
print(fscores,mscores)

{'neg': 0.097, 'neu': 0.728, 'pos': 0.175, 'compound': 1.0} {'neg': 0.1, 'neu': 0.723, 'pos': 0.177, 'compound': 1.0}


# TFIDF

In [33]:
from langdetect import detect
from langdetect import LangDetectException
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
#stemming the words
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_nested_list(nested_list):
    return [
        [
            ' '.join(stemmer.stem(word) for word in sentence.split())
            for sentence in sublist
        ]
        for sublist in nested_list
    ]
menParaOriginal = menPara
womenParaOriginal = womenPara
menPara = stem_nested_list(menPara)
womenPara = stem_nested_list(womenPara)
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))

In [35]:
tobeTFIDF = [" ".join(" ".join(row) for row in menPara)," ".join(" ".join(row) for row in womenPara)]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tobeTFIDF)
feature_names = vectorizer.get_feature_names_out()
dense_tfidf = tfidf_matrix.toarray()

In [36]:
#tfidf of female words
doc_index = 1
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)
top_300_words_fs = sorted_word_scores[:300]
top_300_words_f = [word for word, score in sorted_word_scores[:300]]

In [37]:
#tfidf of male words
doc_index = 0  
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs_m = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores_m = sorted(word_score_pairs_m, key=lambda x: x[1], reverse=True)
top_300_words_ms = sorted_word_scores_m[:300]
top_300_words_m = [word for word, score in sorted_word_scores_m[:300]]

In [38]:
#remove common words and gendered keywords
common = list(set(top_300_words_f) & set(top_300_words_m))
top_300_words_f = [word for word in top_300_words_f if word not in common]
top_300_words_f = [word for word in top_300_words_f if word not in womenWordsNoSpace]
female_300 = " ".join(top_300_words_f)
top_300_words_m = [word for word in top_300_words_m if word not in common]
top_300_words_m = [word for word in top_300_words_m if word not in menWordsNoSpace]
male_300 = " ".join(top_300_words_m)

In [39]:
print(female_300)
print(male_300)

stevenson children nona rosi door eat sit wear oper partner drive ex past cook dress someon student buy guess number point stand correct bad exampl felt hair kid near seem black booli care expect form law pictur tomorrow verb draw drink enjoy femal hard hate st today week worri eight maori babi beauti bu cours danc ed exam food forsyth lesson niec ok second seen tonight angela selena choos cloth continu convers eddi experi eye fall french fun grandmoth half hurt
arthur lawrenc princ uncl dead sword bediver god modr lucan knight age need dream kill shall water montagu bodi morn yet upon bernard charley summer cri tybalt teacher john studi gawain order poem took becom big light martin noth open track act earth jesu labang pass ring seven show yeah ala hermit arm head hold lam land line met bring host howard messag servant epic spear bed escalu field gave luther meet quickli scene set side stay street unto york alaska


In [40]:
from collections import Counter
import re
mwords = male_300.split()
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)

In [41]:
fwords = female_300.split()
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)

In [42]:
from collections import Counter
import re

# Example word lists (assuming male_300 and female_300 are defined as strings)
mwords = male_300.split()  # Male words
fwords = female_300.split()  # Female words

# Filtered words for male
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)
print("Top 40 Male Words:", topMwords)

# Filtered words for female
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)
print("Top 40 Female Words:", topFwords)


Top 40 Male Words: [('arthur', 1), ('lawrenc', 1), ('princ', 1), ('uncl', 1), ('dead', 1), ('sword', 1), ('bediver', 1), ('god', 1), ('modr', 1), ('lucan', 1), ('knight', 1), ('age', 1), ('need', 1), ('dream', 1), ('kill', 1), ('shall', 1), ('water', 1), ('montagu', 1), ('bodi', 1), ('morn', 1), ('yet', 1), ('upon', 1), ('bernard', 1), ('charley', 1), ('summer', 1), ('cri', 1), ('tybalt', 1), ('teacher', 1), ('john', 1), ('studi', 1), ('gawain', 1), ('order', 1), ('poem', 1), ('took', 1), ('becom', 1), ('big', 1), ('light', 1), ('martin', 1), ('noth', 1), ('open', 1)]
Top 40 Female Words: [('stevenson', 1), ('children', 1), ('nona', 1), ('rosi', 1), ('door', 1), ('eat', 1), ('sit', 1), ('wear', 1), ('oper', 1), ('partner', 1), ('drive', 1), ('ex', 1), ('past', 1), ('cook', 1), ('dress', 1), ('someon', 1), ('student', 1), ('buy', 1), ('guess', 1), ('number', 1), ('point', 1), ('stand', 1), ('correct', 1), ('bad', 1), ('exampl', 1), ('felt', 1), ('hair', 1), ('kid', 1), ('near', 1), ('se

# random forest

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)


proba = clf.predict_proba(X)[:, 1]  # Probability of being female
female_contributions = np.dot(X.T.toarray(), proba)
female_importance = female_contributions / np.sum(X.toarray(), axis=0)
female_importance_df = pd.DataFrame({
    "Word": feature_names,
    "FemaleImportance": female_importance
}).sort_values(by="FemaleImportance", ascending=False)

print("Top words most related to Female:")
print(female_importance_df.head(10))

Top words most related to Female:
         Word  FemaleImportance
2900     issu               1.0
4789  rooster               1.0
201    angela               1.0
4246  phillip               1.0
4982   selena               1.0
1670    eight               1.0
5776     tina               1.0
3861     nona               1.0
4797     rosi               1.0
3491  marriag               1.0


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)

proba = clf.predict_proba(X)[:, 0]  # Probability of being male (class 0)


male_contributions = np.dot(X.T.toarray(), proba)
male_importance = male_contributions / np.sum(X.toarray(), axis=0)
male_importance_df = pd.DataFrame({
    "Word": feature_names,
    "MaleImportance": male_importance
}).sort_values(by="MaleImportance", ascending=False)

print("Top words most related to Male:")
print(male_importance_df.head(10))

Top words most related to Male:
          Word  MaleImportance
2193    gawain             1.0
5300     spare             1.0
633       boat             1.0
6202      waig             1.0
220        ant             1.0
6198    wagner             1.0
2509    hermit             1.0
5452  strength             1.0
1408   destini             1.0
3091    knight             1.0


In [45]:
fm = ', '.join(map(str, [item for sublist in menParaOriginal for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenParaOriginal for item in sublist]))
ps = PorterStemmer()
words = word_tokenize(fw)
matching = [word for word in words if ps.stem(word) in {"cass", "aborigin","sungen"}]
matching

['sungen',
 'sungen',
 'sungen',
 'aboriginal',
 'aboriginal',
 'aboriginal',
 'aboriginal',
 'cass',
 'cass',
 'cass',
 'cass']